# Build a semantic search engine

This tutorial will familiarize you with LangChain's document loader, embedding, and vector store abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation, or RAG (see our RAG tutorial here).

Here we will build a search engine over a PDF document. This will allow us to retrieve passages in the PDF that are similar to an input query.

## Prerequisites

You will need to provision the following Azure resources:
* Azure OpenAI with two models: GPT-4o and Embedding model.
* Azure AI Search.
You can run the terraform template from folder `../400_azure_ai_foundry` to create all of these resources by simply running the following commands.

```sh
terraform init
terraform plan -out tfplan
terraform apply tfplan
```

Then you will need to setup the values for the environment variables in file `.env`.

The following resources should be created.

![](images/resources.png)

## Documents and Document Loaders

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has three attributes:

* page_content: a string representing the content;
* metadata: a dict containing arbitrary metadata;
* id: (optional) a string identifier for the document.

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

This tutorial requires the langchain-community and pypdf packages.

In [1]:
%pip install langchain-community pypdf --quiet

Note: you may need to restart the kernel to use updated packages.


## Loading documents

Let's load a PDF into a sequence of Document objects. We can consult the LangChain documentation for available PDF document loaders. Let's select PyPDFLoader, which is fairly lightweight.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/azure-for-architects.pdf" #nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

701


PyPDFLoader loads one Document object per PDF page. For each, we can easily access:

* The string content of the page;
* Metadata containing the file name and page number.

In [11]:
print(f"{docs[200].page_content}\n")
print(docs[200].metadata)

168 | Cost management for Azure solutions
It is important for companies to keep a close watch on Azure consumption and usage. 
Although they can create policies to set organizational standards and conventions, 
there is also a need to keep track of billing and consumption data. Moreover, they 
should employ best practices for consuming Azure resources so that the return is 
maximized. For this, architects need to know about Azure resources and features, their 
corresponding costs, and the cost/benefit analysis of features and solutions.
In this chapter, we will cover the following topics:
• Azure offer details
• Billing
• Invoicing
• Usage and quotas
• Usage and Billing APIs
• Azure pricing calculator
• Best practices for cost optimization
Let's go ahead and discuss each of these points.
Azure offer details
Azure has different offers that you can purchase. So far, we have discussed pay-as-
you-go, but there are others, such as Enterprise Agreements (EAs), Azure Sponsorship, 
and Azure 

## Splitting text by characters

For both information retrieval and downstream question-answering purposes, a page may be too coarse a representation. Our goal in the end will be to retrieve `Document` objects that answer an input query, and further splitting our PDF will help ensure that the meanings of relevant portions of the document are not "washed out" by surrounding text.

We can use text splitters for this purpose. Here we will use a simple text splitter that partitions based on characters. We will split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it. We use the RecursiveCharacterTextSplitter, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We set `add_start_index=True` so that the character index where each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print("Number of splits: ", len(all_splits))

Number of splits:  1385


View a sample split or chunk.

In [27]:
# print a sample chunk
print(all_splits[309])
print("---------------")
print(all_splits[310])

page_content='increased, automation has found increased focus and attention. The primary reason 
is that cloud technology's agility and flexibility provide an opportunity to provision, 
deprovision, and manage these resources on the fly in a tiny fraction of the time it 
used to take. Along with this flexibility and agility come the requirements to be more 
predictable and consistent with the cloud because it has become easy for organizations 
to create resources.
Provisioning
• Data centers
• PaaS services
• Application 
deployments
• Environment 
and application 
Operations
• Backups and 
restores
• Monitoring
• Performance
• Disaster 
recovery
• Availability
• Scalability
• Stop-start-
pause-resume
Management
• Access 
management
• Resource 
locking
• Policies
• Tagging
• Cost
• releases
Deprovisioning
• Delete and 
tear down 
resources
• Soft delete 
of application 
• Transfer' metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.0 (Windows)', 'creationdate

## Creating Embeddings

`Vector search` is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can embed it as a vector of the same dimension and use vector similarity metrics (such as cosine similarity) to identify related text.

`LangChain` supports embeddings from dozens of providers. These models specify how text should be converted into a numeric vector. 

In [15]:
%pip install -qU langchain-openai --quiet
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.



In [33]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv

if os.path.exists(".env"):
    load_dotenv(override=True)


embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_MODEL"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
)

In [18]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])
print(vector_2[:10])

Generated vectors of length 3072

[0.010515968315303326, -0.01905210129916668, -0.017833741381764412, 0.022737640887498856, 0.008901641704142094, -0.006392581854015589, -0.011467811651527882, 0.024169212207198143, -0.0178185123950243, 0.02282901667058468]
[-0.002384420484304428, -0.014230713248252869, -0.017530273646116257, 0.012981644831597805, 0.021096978336572647, -0.03303893655538559, -0.021400220692157745, 0.027956021949648857, -0.031335003674030304, 0.007270587608218193]


Armed with a model for generating text embeddings, we can next store them in a special data structure that supports efficient similarity search.

## Vector stores for Azure AI Search

`LangChain VectorStore` objects contain methods for adding text and `Document` objects to the store, and querying them using various similarity metrics. They are often initialized with embedding models, which determine how text data is translated to numeric vectors.

`LangChain` includes a suite of integrations with different vector store technologies. Some vector stores are hosted by a provider (e.g., various cloud providers) and require specific credentials to use; some (such as `Postgres`) run in separate infrastructure that can be run locally or via a third-party; others can run in-memory for lightweight workloads.

In [19]:
%pip install --upgrade --quiet azure-search-documents
%pip install --upgrade --quiet azure-identity

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
from langchain_openai import AzureOpenAIEmbeddings, AzureOpenAIEmbeddings

embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_MODEL"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

## Create vector store instance

Create instance of the `AzureSearch` class using the embeddings from above.

In [38]:
from langchain_community.vectorstores.azuresearch import AzureSearch

# Specify additional properties for the Azure client such as the following https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/core/azure-core/README.md#configurations
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"],
    azure_search_key=os.environ["AZURE_SEARCH_SERVICE_ADMIN_KEY"],
    index_name=os.environ["AZURE_SEARCH_SERVICE_INDEX"],
    embedding_function=embeddings.embed_query,
    # Configure max retries for the Azure client
    additional_search_client_options={"retry_total": 3},
    relevance_score_fn="cosine",
)

## Insert text and embeddings into vector store

This step loads, chunks, and vectorizes the sample document, and then indexes the content into a search index on `Azure AI Search`.

In [39]:
vector_store.add_documents(documents=all_splits)

TypeError: azure.search.documents._generated.operations._documents_operations.DocumentsOperations.index() got multiple values for keyword argument 'error_map'

Once we've instantiated a VectorStore that contains documents, we can query it. VectorStore includes methods for querying:

* Synchronously and asynchronously;
* By string query and by vector;
* With and without returning similarity scores;
* By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).

## Perform a vector similarity search with relevance scores

Execute a pure vector similarity search using the `similarity_search_with_relevance_scores()` method. Queries that don't meet the threshold requirements are exluded.

In [ ]:
from pprint import pprint

docs_and_scores = vector_store.similarity_search_with_relevance_scores(
    query="Who are the authors of the book Azure for Architects ?",
    k=4,
    score_threshold=0.70,
)

pprint(docs_and_scores)

## Perform a hybrid search

`Hybrid search` combines keyword and semantic similarity, marrying the benefits of both approaches.
Execute hybrid search using the `search_type` or `hybrid_search()` method. Vector and nonvector text fields are queried in parallel, results are merged, and top matches of the unified result set are returned.

In [ ]:
# Perform a hybrid search using the search_type parameter
docs = vector_store.similarity_search(
    query="Who are the authors of the book Azure for Architects",
    k=3,
    search_type="hybrid",
)
print(docs[0].page_content)

## Retrieval and Generation using LangGraph

Now let’s write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

For generation, we will use the chat model selected at the start of the tutorial.

We’ll use a prompt for `RAG` that is checked into the `LangChain` prompt hub.

In [ ]:
%pip install --quiet --upgrade langgraph

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain import hub

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [ ]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

question = "Who are the authors of the book Azure for Architects and when it was published ?"

retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(prompt)

print(answer.content)

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain import hub
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

response = graph.invoke({"question": "Who are the authors of the book Azure for Architects and when it was published ?"})

print(response["answer"])

LangGraph also comes with built-in utilities for visualizing the control flow of your application.

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))